# 지난 시간 결과
0.49정도의 결과를 가졌다
--> 정말 대충 짠 코드인데 절반을 맞췄다는게 신기하다

# Feedback
1) feature engineering
--> 어떤 feature들이 주요한 역할을 하는가?

2) 데이터 정규화
--> min-max, stardarization

3) MLP 
--> 다른 알고리즘들은 어떤게 있을까?
--> 파이토치로 구성해보는 건 어떨까?

4) threshold
--> 1.5를 기준으로해서 잡았는데, 이것은 무엇을 기준으로 잡을 수 있을까?


# Step1: Feature engineering
1) object변수들을 원핫인코딩으로 바꿔주겠다.
<br>
2) 좋은 Feature이 무엇인지 search해보자<br>
<i style ="border: 1px solid black;"> Feature Engineering이란 머신러닝 문제에 대해서 타겟에 대해서 최고로 잘 표현할 수 있게 가공되지 않은 데이터를 가공하는 것을 의미한다. <i>
<br>
1. You can isolate and highlight key information, which helps your algorithms “focus” on what’s important.
중요하다고 생각하는 변수를 강조시킬 수 있다.

2. You can bring in your own domain of expertise.
자신만의 도메인을 도입할 수 있다.

3. Most importantly, once you understand the “vocabulary” of feature engineering, you can bring in other people’s domain expertise!
<br>https://data-newbie.tistory.com/93

In [109]:
import pandas as pd
# 데이터 불러오기
train = pd.read_csv("/Users/igyuseog/Desktop/open data/train.csv")
test = pd.read_csv("/Users/igyuseog/Desktop/open data/test_x.csv")

# 첫번째 열에 해당하는 (index)를 지웠다.
train = train.iloc[:,1:]
test = test.iloc[:,1:]

#출력
train.head()
test.head()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,736,2.0,2941,3.0,4621,1.0,4857,2.0,2550,...,0,1,0,0,1,0,1,0,1,1
1,3.0,514,2.0,1952,3.0,1552,3.0,821,4.0,1150,...,0,0,0,0,0,0,0,0,0,0
2,3.0,500,2.0,2507,4.0,480,2.0,614,2.0,1326,...,0,1,0,1,1,0,1,0,1,1
3,1.0,669,1.0,1050,5.0,1435,2.0,2252,5.0,2533,...,1,1,1,1,1,1,1,1,1,1
4,2.0,499,1.0,1243,5.0,845,2.0,1666,2.0,925,...,1,1,0,1,1,0,1,1,1,1


In [110]:
#object인 column명을 알 수 있다.
object_col = train.columns[train.dtypes == "object"].tolist()

#groupby (다양한 기능들이 존재하는데, 내가 원했던건 그룹별로 voted의 양상이었다.)
train.groupby(["gender"])["voted"].value_counts()


# --------------------------------------------------------

#step1) drop --> 필요없다고 판단되는 것들을 지웠다.
y = train["voted"]
train = train.drop(["engnat","hand","race","religion","urban","voted"], axis = 1)
test = test.drop(["engnat","hand","race","religion","urban"], axis = 1)

#step2)one_hot encoding은 아래와 같이 get_dumpy를 통해 가능하다.
train = pd.get_dummies(train)
test = pd.get_dummies(test)

#step3) 정규화 min-max
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
train = min_max_scaler.fit_transform(train)
test = min_max_scaler.fit_transform(test)

# train & test

In [83]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes = [10,10,10,10,10]).fit(train,y)

In [105]:
import numpy as np

pred = model.predict(train)
pred = np.where(pred < 1.483,1,2)

acc = sum(pred == y) / len(pred)
print("accuracy = ", acc)

accuracy =  0.6836730211719231


In [112]:
submit = pd.read_csv("/Users/igyuseog/Desktop/open data/sample_submission.csv")
pred = model.predict(test)
pred = np.where(pred < 1.483,1,2)

submit["voted"] = pred

In [123]:
submit.loc[:,"voted"].to_csv("/Users/igyuseog/Desktop/open data/MLP2.csv")

# Dacon 75위 참고자료
https://dacon.io/competitions/official/235647/codeshare/1819?page=1&dtype=recent&ptype=pub

In [1]:
import random
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [5]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
DEVICE

'cpu'

In [13]:
train = pd.read_csv("/Users/igyuseog/Desktop/open data/train.csv")
train = train.drop(train[train["familysize"] > 50].index)
test = pd.read_csv("/Users/igyuseog/Desktop/open data/test_x.csv")


(45532, 78)
(45529, 78)


In [34]:
drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE',
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE',
             'index', 'hand']
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}

train_y = train["voted"]
train_x = train.drop(drop_list + ["voted"], axis = 1)
test_x = test.drop(drop_list, axis = 1)

#타입 변경
train_x = train_x.astype(replace_dict)
test_x = test_x.astype(replace_dict)
#원핫인코딩
train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)
print(train_x.head)
#numpy
train_y = 2 - train_y.to_numpy() #0이 투표한다. 1이 투표 안한다 
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()

<bound method NDFrame.head of        QaA  QbA  QcA  QdA  QeA  QfA  QgA  QhA  QiA  QjA  ...  \
0      3.0  4.0  5.0  1.0  2.0  5.0  2.0  4.0  5.0  4.0  ...   
1      5.0  5.0  3.0  5.0  1.0  3.0  1.0  1.0  5.0  3.0  ...   
2      4.0  1.0  1.0  4.0  5.0  1.0  4.0  1.0  3.0  2.0  ...   
3      3.0  3.0  4.0  3.0  1.0  2.0  4.0  3.0  5.0  4.0  ...   
4      1.0  1.0  5.0  2.0  1.0  2.0  1.0  1.0  5.0  5.0  ...   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
45527  2.0  5.0  4.0  1.0  1.0  1.0  1.0  1.0  1.0  4.0  ...   
45528  2.0  3.0  4.0  1.0  3.0  2.0  2.0  1.0  2.0  5.0  ...   
45529  4.0  1.0  1.0  4.0  5.0  4.0  5.0  1.0  5.0  1.0  ...   
45530  1.0  3.0  4.0  2.0  1.0  1.0  1.0  1.0  5.0  1.0  ...   
45531  3.0  5.0  5.0  3.0  1.0  4.0  3.0  3.0  4.0  4.0  ...   

       religion_Christian_Protestant  religion_Hindu  religion_Jewish  \
0                                  0               0                0   
1                                  0               1   

In [38]:
# 정규화
train_x[:,:20] = (train_x[:,:20] -3.) / 2.
test_x[:,:20] = (test_x[:,:20] -3.) / 2.

train_x[:,20] = (train_x[:,20] -5.) / 5.
test_x[:,20] = (test_x[:,20] -5.) / 5.

train_x[:,21:31] = (train_x[:, 21:31] - 3.5) / 3.5
test_x[:,21:31] = (test_x[:, 21:31] - 3.5) / 3.5
# 여기서 tensor로 바꿔주는 구나.
train_y = torch.tensor(train_y, dtype = torch.float32)
train_x = torch.tensor(train_x, dtype = torch.float32)
test_x = torch.tensor(test_x, dtype = torch.float32)
train_len, test_len = len(train_x), len(test_x)

<ipython-input-38-71b0a20b823f>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_y = torch.tensor(train_y, dtype = torch.float32)
<ipython-input-38-71b0a20b823f>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_x = torch.tensor(train_x, dtype = torch.float32)
<ipython-input-38-71b0a20b823f>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_x = torch.tensor(test_x, dtype = torch.float32)


In [39]:
N_MODEL = 10
N_EPOCH = 50
BATCH_SIZE = 128
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}

prediction = np.zeros((11383,1), dtype = np.float32)

In [41]:
for no in range(N_MODEL):
    train_loader = DataLoader(TensorDataset(train_x,train_y),
                             shuffle = True, drop_last = True, **LOADER_PARAM)
    test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype = torch.float32)),
                            shuffle = False, drop_last = False, **LOADER_PARAM)
    model = nn.Sequential(
        nn.Dropout(0.06),
        nn.Linear(91,96, bias = False),
        nn.LeakyReLU(0.05, inplace= True),
        
        nn.Dropout(0.5),
        nn.Linear(96,36, bias = False),
        nn.ReLU(inplace= True),
        
        nn.Linear(36,1)
    ).to(DEVICE)
    
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight = torch.tensor([1.20665], device = DEVICE))
    optimizer = optim.AdamW(model.parameters(), lr = 1e-3, weight_decay = 4e-2)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0= N_EPOCH // 4, eta_min = 1.2e-5)
    
    

In [43]:
model.train()
for epoch in tqdm(range(N_EPOCH), desc = '{:02d} / {:02d}'.format(no + 1, N_MODEL)):
    for idx, (xx,yy) in enumerate(train_loader):
        optimizer.zero_grad()
        
        xx,yy = xx.to(DEVICE) , yy.to(DEVICE)
        pred = model(xx).squeeze()
        loss = criterion(pred, yy)
        loss.backward()
        optimizer.step()
        scheduler.step(epoch + idx / len(train_loader))
        

10 / 10: 100%|██████████| 50/50 [01:11<00:00,  1.43s/it]


In [44]:
model.eval()
with torch.no_grad():
    for idx, (xx,_) in enumerate(test_loader):
        xx = xx.to(DEVICE)
        pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
        prediction[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                += pred[:, :] / N_MODEL

In [45]:
prediction

array([[0.15166652],
       [0.18116131],
       [0.14082035],
       ...,
       [0.12067059],
       [0.12988257],
       [0.16335401]], dtype=float32)

In [46]:
import random
from datetime import datetime

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE',
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE',
             'index', 'hand']
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}
train_data = pd.read_csv("/Users/igyuseog/Desktop/open data/train.csv")
test_data = pd.read_csv("/Users/igyuseog/Desktop/open data/test_x.csv")

train_data = train_data.drop(train_data[train_data.familysize > 50].index)
train_y = train_data['voted']
train_x = train_data.drop(drop_list + ['voted'], axis=1)
test_x = test_data.drop(drop_list, axis=1)
train_x = train_x.astype(replace_dict)
test_x = test_x.astype(replace_dict)
train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)
train_y = 2 - train_y.to_numpy()
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()

train_y_t = torch.tensor(train_y, dtype=torch.float32)
train_x_t = torch.tensor(train_x, dtype=torch.float32)
test_x_t = torch.tensor(test_x, dtype=torch.float32)
train_x_t[:, :20] = (train_x_t[:, :20] - 3.) / 2.
test_x_t[:, :20] = (test_x_t[:, :20] - 3.) / 2
train_x_t[:, 20] = (train_x_t[:, 20] - 5.) / 4.
test_x_t[:, 20] = (test_x_t[:, 20] - 5.) / 4.
train_x_t[:, 21:31] = (train_x_t[:, 21:31] - 3.5) / 3.5
test_x_t[:, 21:31] = (test_x_t[:, 21:31] - 3.5) / 3.5
test_len = len(test_x_t)

N_REPEAT = 5
N_SKFOLD = 7
N_EPOCH = 48
BATCH_SIZE = 72
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((test_len, 1), dtype=np.float32)

for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(train_x, train_y)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(TensorDataset(train_x_t[train_idx, :], train_y_t[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(train_x_t[valid_idx, :], train_y_t[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(test_x_t, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        model = nn.Sequential(
            nn.Dropout(0.05),
            nn.Linear(91, 180, bias=False),
            nn.LeakyReLU(0.05, inplace=True),
            nn.Dropout(0.5),
            nn.Linear(180, 32, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1)
        ).to(DEVICE)
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=7.8e-2)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=N_EPOCH // 6, eta_min=4e-4)
        prediction_t, loss_t = np.zeros((test_len, 1), dtype=np.float32), 1.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad()
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze()
                loss = criterion(pred, yy)
                loss.backward()
                optimizer.step()
                scheduler.step(epoch + idx / len(train_loader))

            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()
                # print('R{:02d} S{:02d} E{:02d} | {:6.4f}, {:5.2f}%'
                #       .format(repeat + 1, skfold + 1, epoch + 1, running_loss / running_count,
                #               running_acc / running_count * 100))

                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.4f}'.format(repeat + 1, tot / N_SKFOLD))


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
01/07:   0%|          | 0/48 [00:00<?, ?it/s]

R1 -> 0.6054


01/07:  73%|███████▎  | 35/48 [01:41<00:37,  2.91s/it]


KeyboardInterrupt: 